In [1]:
CORPUS_ARXIU = "test_docs_anotat.conllu"

# Importem la llibreria CONLLU
import conllu, pickle, csv
from itertools import zip_longest
from conllu import parse
from io import open

from utils import EntitatNomenada

In [2]:
# Llegim les variables resultants d'executar les regex.
with open('entitats_nomenades.corpus_raw.pickle', 'rb') as file:
    entitats_nomenades_res = pickle.load(file)

with open('tokens.corpus_raw.pickle', 'rb') as file:
    tokens_res = pickle.load(file)

In [3]:
# Guardem el corpus de test anotat manualment a la variable dades
with open(CORPUS_ARXIU, "r", encoding="utf-8") as dades:

    # Llegeix els continguts del fitxer CONLLU i els guarda a la variable anotacions 
    anotacions = dades.read()

# Segmentem en frases tot el corpus i les guardem a la variable frases
frases = conllu.parse(anotacions)

In [4]:
# Obtenim els tokens a partir de l'arxiu de test.
tokens_test = []
for frase in frases:
    for token in frase:
        tokens_test.append((token))

In [5]:
# Comparem els dos conjunts de tokens per garantir que les dades son consistents.
if tokens_res == list(map(str, tokens_test)):
    print("Tot perfecte.")

Tot perfecte.


In [6]:
# Retorna l'etiqueta de la EN i el tipus ('B', 'I', 'O').
def obtenir_tipus_entitat(token):

    # Si no té dades, retornem buit.
    if token['misc'] is None:
        return 'O', None

    # Iterem per les claus del diccionari de dades.
    for clau in token['misc']:

        if clau[0:2] == 'B-':
            return 'B', clau[2:]
        
        if clau[0:2] == 'I-':
            return 'I', clau[2:]

    return 'O', None


In [7]:
# Iterem per tots els tokens cercant EN.
i = 0
entitats_nomenades_test = []
while i < len(tokens_test):

    # Obtenim el token corresponent.
    token_i = tokens_test[i]
    estat, etiqueta = obtenir_tipus_entitat(token_i)
    inici = i

    # Ja hem accedit al token, passem al següent ja en cas d'entrar
    # al bloc condicional.
    i += 1
    
    # Si marca l'inici d'una EN, entrem al bloc.
    if estat == 'B':

        # Obtenim el token corresponent.
        token_i = tokens_test[i]
        estat, _ = obtenir_tipus_entitat(token_i)
        final = i

        # Mentre la EN continuï o no s'hagi acabat el document.
        while estat == 'I' and i < len(tokens_test):

            # Saltem al següent token.
            i += 1

            # Actualitzem la informació.
            token_i = tokens_test[i]
            estat, _ = obtenir_tipus_entitat(token_i)
            final = i

        # Creem la instància d'EntitatNomenada i l'afegim a l'estructura.
        entitats_nomenades_test.append(EntitatNomenada(etiqueta, inici, final - inici, tokens_res[inici:final]))


In [8]:
# Ordenem les llistes pel seu index de token.
entitats_nomenades_res.sort(key=lambda x: x.inici)
entitats_nomenades_test.sort(key=lambda x: x.inici)

In [9]:
for en1, en2 in zip_longest(entitats_nomenades_res, entitats_nomenades_test):

    print("RES:", end=" ")
    if en1 is None:
        print("-")
    else:
        en1.print()

    print("TEST:", end=" ")
    if en2 is None:
        print("-")
    else:
        en2.print()

RES: PER (29-30) = ['Kerman']
TEST: LOC (29-30) = ['Kerman']
RES: PER (31-34) = ['Mohammad', 'Ali', 'Karimi']
TEST: PER (31-34) = ['Mohammad', 'Ali', 'Karimi']
RES: PER (42-43) = ['Kerman']
TEST: LOC (42-43) = ['Kerman']
RES: TIME (64-65) = ['després']
TEST: TIME (64-65) = ['després']
RES: PER (95-97) = ['Shah', 'Bajsh']
TEST: ORG (95-97) = ['Shah', 'Bajsh']
RES: TIME (125-126) = ['ahir']
TEST: TIME (125-126) = ['ahir']
RES: TIME (127-129) = ['un', 'dia']
TEST: TIME (127-129) = ['un', 'dia']
RES: TIME (129-130) = ['després']
TEST: TIME (129-130) = ['després']
RES: ORG (138-141) = ['Ministeri', "d'", 'Informació']
TEST: ORG (138-141) = ['Ministeri', "d'", 'Informació']
RES: ORG (148-149) = ['Govern']
TEST: ORG (148-149) = ['Govern']
RES: ORG (184-186) = ['France', 'Presse']
TEST: ORG (184-186) = ['France', 'Presse']
RES: LOC (211-212) = ['Iran']
TEST: LOC (211-212) = ['Iran']
RES: PER (213-215) = ['Ali', 'Khamenei']
TEST: PER (213-215) = ['Ali', 'Khamenei']
RES: TIME (239-240) = ['diari

In [10]:
# Transformem les instàncies d'EntitatNomenada a tuples per poder operar-hi fàcilment.
entitats_nomenades_res_tuples = set(map(lambda x: x.a_tupla(), entitats_nomenades_res))
entitats_nomenades_test_tuples = set(map(lambda x: x.a_tupla(), entitats_nomenades_test))

# Operem amb els conjunts per obtenir la intersecció i subtraccions.
true_positives = entitats_nomenades_res_tuples & entitats_nomenades_test_tuples
false_positives = entitats_nomenades_res_tuples - entitats_nomenades_test_tuples
false_negatives = entitats_nomenades_test_tuples - entitats_nomenades_res_tuples

# Obtenim les mètriques de cada categoria.
metrica_tp = len(true_positives)
metrica_fp = len(false_positives)
metrica_fn = len(false_negatives)

# Obtenim les mètriques complexes.
precisio = metrica_tp / (metrica_tp + metrica_fp)
reclam = metrica_tp / (metrica_tp + metrica_fn)
f1 = 2 * precisio * reclam / (precisio + reclam)

print("Autèntics positius:", metrica_tp)
print("Falsos positius:", metrica_fp)
print("Falsos negatius:", metrica_fn)
print("=========")
print("Precisió: {}%".format(int(precisio * 10000) / 100))
print("Reclam: {}%".format(int(reclam * 10000) / 100))
print("Mesura F: {}%".format(int(f1 * 10000) / 100))


Autèntics positius: 1333
Falsos positius: 288
Falsos negatius: 296
Precisió: 82.23%
Reclam: 81.82%
Mesura F: 82.03%


In [11]:
for en in false_negatives:
    print(en)

('CARDINAL', 5948, 2, ('100', 'dies'))
('PER', 5843, 1, ('Ardanza',))
('PER', 5375, 1, ('VIP',))
('TIME', 11341, 1, ('desembre',))
('LOC', 10731, 1, ('Brest',))
('PER', 702, 1, ('Llaràs',))
('PER', 11168, 1, ('Belot',))
('LOC', 18648, 1, ('Model',))
('LOC', 14610, 1, ('Carnota',))
('ORG', 19723, 2, ('La', 'Caixa'))
('ORG', 13378, 1, ('Pimecsefes',))
('ORG', 19770, 2, ('La', 'Kutxa'))
('LOC', 16648, 1, ('Barcelona',))
('MISC', 10551, 7, ('Encesa', 'd', 'el', 'signe', 'd', 'el', 'zodíac-Cranc'))
('CARDINAL', 3277, 1, ('400.000',))
('CARDINAL', 2797, 1, ('2.000.000',))
('ORG', 18200, 5, ('Fundació', 'Prudenci', 'Bertrana', 'de', 'Girona'))
('MISC', 6606, 1, ('PAAU',))
('MISC', 5037, 2, ('Ona', 'Catalana'))
('LOC', 5896, 1, ('Donington',))
('ORG', 13996, 1, ('Xfera',))
('LOC', 1293, 2, ('Universitat', 'Pontifícia'))
('MISC', 6641, 1, ('PAAU',))
('ORG', 14745, 5, ('Agrupació', 'de', 'Voluntaris', 'de', 'Reus'))
('LOC', 9463, 3, ('Hospital', 'do', 'Meixoeiro'))
('ORG', 12040, 1, ('Cedel',))


In [12]:
for en in false_positives:
    print(en)

('ORG', 11309, 1, ('Newark',))
('ORG', 19642, 1, ('Tobago',))
('ORG', 4612, 7, ('Comissió', 'de', 'Política', 'Social', 'd', 'el', 'Parlament'))
('PER', 7851, 1, ('Verdaguer',))
('MISC', 16433, 3, ('Seva', ',', 'Matadepera'))
('PER', 13961, 1, ('Saba',))
('MISC', 14610, 1, ('Carnota',))
('MISC', 3222, 2, ('Le', 'Pen'))
('PER', 3933, 5, ('Merril', 'Lynch', 'Mercury', 'Asset', 'Management'))
('LOC', 1426, 3, ('Hebe', 'de', 'Bonafini'))
('ORG', 11452, 1, ('Bayer',))
('ORG', 15511, 4, ('Universitat', 'Protestant', 'de', 'Teologia'))
('ORG', 16356, 2, ('Àsia', 'Menor'))
('LOC', 8034, 1, ('Atlàntida',))
('LOC', 11498, 1, ('Lipobay',))
('PER', 18600, 1, ('cerverí@catradio.com',))
('MISC', 18342, 2, ('Quartet', 'Anònim'))
('TIME', 2881, 1, ('abans',))
('PER', 16891, 1, ('Linepithema',))
('PER', 3930, 2, ('Morgan', 'Stanley'))
('TIME', 12933, 2, ('un', 'dia'))
('PER', 29, 1, ('Kerman',))
('ORG', 8061, 1, ('Comillas',))
('LOC', 13610, 4, ('Delegació', 'd', 'el', 'Govern'))
('PER', 42, 1, ('Kerma

In [13]:
# Obtenim la llista d'etiquetes.
etiquetes_disponibles = set(map(lambda e: e[0], entitats_nomenades_test_tuples))
etiquetes_spacy_org = {'LOC', 'MISC', 'ORG', 'PER'}
etiquetes_propies = set(e for e in etiquetes_disponibles if e not in etiquetes_spacy_org)

In [14]:
# Obtenim les mètriques pel sistema original de spacy.
en_resultat_spacy = set(filter(lambda e: e[0] in etiquetes_spacy_org, entitats_nomenades_res_tuples))
en_test_spacy = set(filter(lambda e: e[0] in etiquetes_spacy_org, entitats_nomenades_test_tuples))

# Operem amb els conjunts per obtenir la intersecció i subtraccions.
true_positives_spacy = en_resultat_spacy & en_test_spacy
false_positives_spacy = en_resultat_spacy - en_test_spacy
false_negatives_spacy = en_test_spacy - en_resultat_spacy

# Obtenim les mètriques de cada categoria.
metrica_tp = len(true_positives_spacy)
metrica_fp = len(false_positives_spacy)
metrica_fn = len(false_negatives_spacy)

# Obtenim les mètriques complexes.
precisio = metrica_tp / (metrica_tp + metrica_fp)
reclam = metrica_tp / (metrica_tp + metrica_fn)
f1 = 2 * precisio * reclam / (precisio + reclam)

print("DETECTADES PER SPACY EN COMPARACIÓ A L'ETIQUETAT ORIGINAL DE SPACY")
print("Autèntics positius:", metrica_tp)
print("Falsos positius:", metrica_fp)
print("Falsos negatius:", metrica_fn)
print("=========")
print("Precisió: {}%".format(int(precisio * 10000) / 100))
print("Reclam: {}%".format(int(reclam * 10000) / 100))
print("Mesura F: {}%".format(int(f1 * 10000) / 100))

DETECTADES PER SPACY EN COMPARACIÓ A L'ETIQUETAT ORIGINAL DE SPACY
Autèntics positius: 829
Falsos positius: 233
Falsos negatius: 236
Precisió: 78.06%
Reclam: 77.84%
Mesura F: 77.95%


In [15]:
# Obtenim les mètriques pel sistema original de spacy.
en_resultat_spacy = set(filter(lambda e: e[0] in etiquetes_spacy_org, entitats_nomenades_res_tuples))
en_test_spacy = entitats_nomenades_test_tuples

# Operem amb els conjunts per obtenir la intersecció i subtraccions.
true_positives_spacy = en_resultat_spacy & en_test_spacy
false_positives_spacy = en_resultat_spacy - en_test_spacy
false_negatives_spacy = en_test_spacy - en_resultat_spacy

# Obtenim les mètriques de cada categoria.
metrica_tp = len(true_positives_spacy)
metrica_fp = len(false_positives_spacy)
metrica_fn = len(false_negatives_spacy)

# Obtenim les mètriques complexes.
precisio = metrica_tp / (metrica_tp + metrica_fp)
reclam = metrica_tp / (metrica_tp + metrica_fn)
f1 = 2 * precisio * reclam / (precisio + reclam)

print("DETECTADES PER SPACY EN COMPARACIÓ A L'ETIQUETAT COMPLET")
print("Autèntics positius:", metrica_tp)
print("Falsos positius:", metrica_fp)
print("Falsos negatius:", metrica_fn)
print("=========")
print("Precisió: {}%".format(int(precisio * 10000) / 100))
print("Reclam: {}%".format(int(reclam * 10000) / 100))
print("Mesura F: {}%".format(int(f1 * 10000) / 100))

DETECTADES PER SPACY EN COMPARACIÓ A L'ETIQUETAT COMPLET
Autèntics positius: 829
Falsos positius: 233
Falsos negatius: 800
Precisió: 78.06%
Reclam: 50.89%
Mesura F: 61.61%


In [16]:
# Iterem per les etiquetes propies per obtenir les mètriques per etiqueta.
for etiqueta in etiquetes_propies:

    # Obtenim les mètriques pel sistema original de spacy.
    en_resultat_etiqueta = set(filter(lambda e: e[0] == etiqueta, entitats_nomenades_res_tuples))
    en_test_etiqueta = set(filter(lambda e: e[0] == etiqueta, entitats_nomenades_test_tuples))

    # Operem amb els conjunts per obtenir la intersecció i subtraccions.
    true_positives_etiqueta = en_resultat_etiqueta & en_test_etiqueta
    false_positives_etiqueta = en_resultat_etiqueta - en_test_etiqueta
    false_negatives_etiqueta = en_test_etiqueta - en_resultat_etiqueta

    # Obtenim les mètriques de cada categoria.
    metrica_tp = len(true_positives_etiqueta)
    metrica_fp = len(false_positives_etiqueta)
    metrica_fn = len(false_negatives_etiqueta)

    # Obtenim les mètriques complexes.
    precisio = metrica_tp / (metrica_tp + metrica_fp)
    reclam = metrica_tp / (metrica_tp + metrica_fn)
    f1 = 2 * precisio * reclam / (precisio + reclam)

    print("= ETIQUETA", etiqueta, "===")
    print("Autèntics positius:", metrica_tp)
    print("Falsos positius:", metrica_fp)
    print("Falsos negatius:", metrica_fn)
    print("=========")
    print("Precisió: {}%".format(int(precisio * 10000) / 100))
    print("Reclam: {}%".format(int(reclam * 10000) / 100))
    print("Mesura F: {}%".format(int(f1 * 10000) / 100))
    print("")

= ETIQUETA PERCENT ===
Autèntics positius: 58
Falsos positius: 0
Falsos negatius: 0
Precisió: 100.0%
Reclam: 100.0%
Mesura F: 100.0%

= ETIQUETA MONEY ===
Autèntics positius: 38
Falsos positius: 0
Falsos negatius: 0
Precisió: 100.0%
Reclam: 100.0%
Mesura F: 100.0%

= ETIQUETA TIME ===
Autèntics positius: 162
Falsos positius: 36
Falsos negatius: 25
Precisió: 81.81%
Reclam: 86.63%
Mesura F: 84.15%

= ETIQUETA CARDINAL ===
Autèntics positius: 101
Falsos positius: 12
Falsos negatius: 29
Precisió: 89.38%
Reclam: 77.69%
Mesura F: 83.12%

= ETIQUETA DATE ===
Autèntics positius: 109
Falsos positius: 1
Falsos negatius: 5
Precisió: 99.09%
Reclam: 95.61%
Mesura F: 97.32%

= ETIQUETA ORDINAL ===
Autèntics positius: 28
Falsos positius: 6
Falsos negatius: 0
Precisió: 82.35%
Reclam: 100.0%
Mesura F: 90.32%

= ETIQUETA QUANTITY ===
Autèntics positius: 8
Falsos positius: 0
Falsos negatius: 1
Precisió: 100.0%
Reclam: 88.88%
Mesura F: 94.11%

